In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.decomposition import PCA
import pickle

In [0]:
def handleMIssingVals(df2):
  df2['ps_ind_02_cat'] = df2['ps_ind_02_cat'].fillna(1.0)
  df2['ps_ind_04_cat']=df2['ps_ind_04_cat'].fillna(0.0)
  df2['ps_ind_05_cat']=df2['ps_ind_05_cat'].fillna(7.0)
  df2['ps_reg_03']=df2['ps_reg_03'].fillna(df2['ps_reg_03'].mean())
  df2['ps_car_01_cat']=df2['ps_car_01_cat'].fillna(11.0)
  df2['ps_car_05_cat']=df2['ps_car_05_cat'].fillna(2.0)
  df2['ps_car_07_cat']=df2['ps_car_07_cat'].fillna(1.0)
  df2['ps_car_09_cat']=df2['ps_car_09_cat'].fillna(2.0)
  df2['ps_car_11']=df2['ps_car_11'].fillna(3.0)
  df2['ps_car_12']=df2['ps_car_12'].fillna(df2['ps_car_12'].mean())
  df2['ps_car_14']=df2['ps_car_14'].fillna(df2['ps_car_14'].mean())

In [0]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/auto.csv')
df=df.replace(-1,np.nan)
handleMIssingVals(df)

In [0]:
target=df['target']

In [0]:
drop_cols=['id', 'ps_car_13' ,'ps_car_14' , 'ps_reg_03' , 'ps_ind_10_bin' , 'ps_ind_11_bin' , 'ps_ind_13_bin' ,'ps_ind_14' , 'ps_car_03_cat','target']
df.drop(drop_cols,axis=1,inplace=True)

In [0]:
cat_cols=df[['ps_car_11_cat','ps_ind_02_cat','ps_ind_04_cat','ps_ind_05_cat','ps_car_01_cat','ps_car_04_cat','ps_car_05_cat','ps_car_06_cat','ps_car_10_cat']]

In [0]:
for eachcol in cat_cols:
  dummies=pd.get_dummies(cat_cols[eachcol],prefix_sep='_',prefix=eachcol)
  df=df.join(dummies)
data_vars=df.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_cols]
df_final=df[to_keep]

In [0]:
target.value_counts()

0    573518
1     21694
Name: target, dtype: int64

In [0]:
rescale_cols=['ps_ind_01' ,'ps_ind_03','ps_ind_15','ps_car_11', 'ps_car_15', 'ps_calc_04', 'ps_calc_05','ps_calc_06','ps_calc_07','ps_calc_08','ps_calc_09','ps_calc_10','ps_calc_11','ps_calc_12','ps_calc_13','ps_calc_14']
scaler=StandardScaler()
scaled_Data=scaler.fit_transform(df_final[rescale_cols])

In [0]:
df_scaled=pd.DataFrame(scaled_Data,columns=rescale_cols)
df_final.drop(rescale_cols,axis=1,inplace=True)
df_final=pd.concat([df_final,df_scaled],axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(df_final, target, test_size=0.3, random_state=0)
columns = X_train.columns

In [0]:
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


length of oversampled data is  802866
Number of no subscription in oversampled data 401433
Number of subscription 401433
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


In [0]:
data_final_vars=df_final.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]



In [0]:
os_data_y['y'].value_counts()

1    401433
0    401433
Name: y, dtype: int64

In [0]:
pca = PCA(n_components=10)
train_pca=pca.fit(os_data_X)
train_dta=pca.transform(os_data_X)
test_dta=pca.transform(X_test)

In [0]:
principalDF=pd.DataFrame(train_dta,columns=['Principal Component1','Principal Component2','Principal Component3','Principal Component4','Principal Component5','Principal Component6','Principal Component7','Principal Component8','Principal Component9','Principal Component10'])

In [0]:
principalDF.shape

(802866, 10)

In [0]:
from sklearn.feature_selection import RFE

In [0]:
rfelgModel=LogisticRegression()
rfe = RFE(rfelgModel, 5)
rfe = rfe.fit(principalDF, os_data_y)
print(rfe.support_)
print(rfe.ranking_)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[False False  True False  True  True False  True  True False]
[6 5 1 3 1 1 4 1 1 2]


In [0]:
lgModel=LogisticRegression()
lgModel.fit(train_dta,os_data_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
lgModel.score(test_dta,y_test)

0.49736789050424496

In [0]:
pred_y=lgModel.predict(test_dta)

In [0]:
print(confusion_matrix(y_test,pred_y))

[[85050 87035]
 [ 2717  3762]]


In [0]:
print(classification_report(y_test,pred_y))

              precision    recall  f1-score   support

           0       0.97      0.49      0.65    172085
           1       0.04      0.58      0.08      6479

    accuracy                           0.50    178564
   macro avg       0.51      0.54      0.37    178564
weighted avg       0.94      0.50      0.63    178564



In [0]:
0.66+0.08

0.74

In [0]:
from sklearn.metrics import f1_score,recall_score,precision_score

In [0]:
print(f1_score(y_test, pred_y, average='micro'))
print(f1_score(y_test, pred_y, average='macro'))
print(f1_score(y_test, pred_y, average='weighted'))

0.49736789050424496
0.3659751600027177
0.6336582685480865


In [0]:
recall_score(y_test,pred_y)

0.5806451612903226

In [0]:
precision_score(y_test,pred_y)

0.0414330869962664

In [0]:
f1_score(y_test,pred_y)

0.07734693038365065

logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [0]:
filename = '/content/drive/My Drive/Colab Notebooks/LogisticRegSMOTE.sav'
pickle.dump(lgModel, open(filename, 'wb'))